In [ ]:
# if you want to try this on MS Azure Notebooks you need to install tifffile first:
!pip install tifffile

In [3]:
# this one needs the module 'tifffile' (pip install tifffile)
import zeisstiffmeta as zm 
    

src = 'test.tif'

# extract the full metadata as defined by Zeiss:
tab = zm.zeiss_meta(src)

# translate into a dict and filter the meta using the zm.vip list
print('#metadata filter list:\n',zm.vip,'\n')
m = zm.meta_to_dict(tab)

# translate all into a dict
mall = zm.meta_to_dict_all(tab)

print('# dictionary of filtered metadata:')
print(m,'\n')

print('# dictionary of all metadata:')
#print(mall)



#metadata filter list:
 ['Detector', 'Store resolution', 'Stage at X', 'Stage at Y', 'Pixel Size', 'EHT', 'I Probe', 'Mag', 'WD', 'Beam Offset X', 'Beam Offset Y', 'Beam Shift Align X', 'Beam Shift Align Y', 'Stage at R', 'Stage at M', 'Stage at T', 'Stage at Z', 'Gun Vacuum', 'System Vacuum', '__filename'] 

# dictionary of filtered metadata:
{'Store resolution': '1024 * 768', 'Detector': 'SE2', 'Mag': 2000.0, 'I Probe': 6.9e-10, 'Beam Offset Y': 0.0, 'Gun Vacuum': 8.76e-10, 'Beam Offset X': 0.0, 'Pixel Size': 5.5820000000000006e-08, 'Beam Shift Align Y': -1.495e-05, 'WD': 0.0077, 'System Vacuum': 9.67e-07, 'Beam Shift Align X': -3.0109999999999998e-05, 'Stage at M': 0.0, 'Stage at Z': 0.039591, 'Stage at Y': 0.0589383, 'Stage at X': 0.0654402, 'EHT': 8000.0, 'Stage at T': 0.0, 'Stage at R': 15.7, '__filename': 'test.tif', 'xpts': 1024, 'ypts': 768, 'width': 5.7159680000000006e-05, 'height': 4.2869760000000005e-05} 

# dictionary of all metadata:


In [ ]:
tab



# read the metadata from all tif files in a directory (and all subdirs)  and feed it to a Pandas dataframe

In [8]:
import pandas as pd
import pathlib

#path to look for files
mypath = 'y:/data import/SEM/'

#p = pathlib.Path.cwd()
p = pathlib.Path(mypath)


flist = p.rglob('*.tif')
data = []
failed = []
for f in flist :
    try:
        tab = zm.zeiss_meta(str(f))
        m = zm.meta_to_dict(tab)
        data.append(m)
        
    except:
        failed.append(f)
        

print('failed tif files: ',failed)
df = pd.DataFrame(data)
df.to_csv('metadata-out.csv') #write the dataframe to a csv file
df.head(5)

failed tif files:  []


,Beam Offset X,Beam Offset Y,Beam Shift Align X,Beam Shift Align Y,Detector,EHT,Gun Vacuum,I Probe,Mag,Pixel Size,...,Stage at Y,Stage at Z,Store resolution,System Vacuum,WD,__filename,height,width,xpts,ypts
0,-0.000004,-0.000010,-7.132700e-07,-1.368000e-05,SE2,5000.0,7.990000e-10,5.000000e-09,102.0,1.095000e-06,...,0.064989,0.032468,1024 * 768,0.000002,0.010532,y:\data import\SEM\2017-01-24 Mirro aligned6.tif,0.000841,0.001121,1024,768
1,0.000000,0.000000,-8.920000e-06,-1.190000e-06,USB TV2,0.0,7.080000e-10,5.000000e-09,696.0,1.604000e-07,...,0.065700,0.012490,1024 * 768,0.000002,0.036210,y:\data import\SEM\2017-01-24 Mirror engaged5.tif,0.000123,0.000164,1024,768
2,0.000000,0.000000,-8.920000e-06,-1.190000e-06,USB TV2,0.0,7.080000e-10,5.000000e-09,696.0,1.604000e-07,...,0.065700,0.012490,1024 * 768,0.000002,0.036210,y:\data import\SEM\2017-01-24 TV1 Mirror4.tif,0.000123,0.000164,1024,768
3,0.000000,0.000000,-8.920000e-06,-1.190000e-06,USB TV2,0.0,7.080000e-10,5.000000e-09,696.0,1.604000e-07,...,0.065700,0.012490,1024 * 768,0.000002,0.036210,y:\data import\SEM\2017-01-24 TV2 Mirror3.tif,0.000123,0.000164,1024,768
4,0.000003,-0.000004,-2.630000e-06,-7.643700e-07,SE2,10000.0,1.260000e-09,1.000000e-08,79.0,1.406000e-06,...,0.065116,0.039554,1024 * 768,0.000001,0.013942,y:\data import\SEM\2017-06-38 CL good focus ho...,0.001080,0.001440,1024,768


In [9]:
#we may do some statistics with pandas:
g = df.groupby('Detector')
g.size()

Detector
ESB         28
InLens     173
SE2        202
USB TV1      8
USB TV2     24
dtype: int64

# convert a tif file so that ImageJ sees the actual scale

In [ ]:
import zeisstiffmeta as zm 

src = 'test.tif'
zm.zeissconvert(src,'test-conv.tif',fullmeta=False)
# The converted file has reduced metadata (vip filter) in the subject field


# a function to convert a full directory tree

In [ ]:
import pathlib # requires python >= 3.4
import zeisstiffmeta as zm 

def zeiss_tiff_dirconvert(basepath,outpath,subdirs=True) :
    '''
    recreates the full directory tree found under basepath in the outpath folder
    '''
    p = pathlib.Path(basepath)
    o = pathlib.Path(outpath)
    if subdirs == True :
        res = p.rglob('*.tif')
    else :
        res = p.glob('*.tif')

    for x in res :
        xx = x.relative_to(p) # rel path with respect to search root
        new = o / xx # 
        print(new)
        if not new.parent.exists() :
            new.parent.mkdir()
        zm.zeissconvert(str(x),str(new))
        
